<div style="background-color:#000;"><img src="pqn.png"></img></div>

These libraries help us work with data, dates, and create visualizations

In [ ]:
import glob
import os
import datetime as dt
import polars as pl
import matplotlib.pyplot as plt

We use these libraries to handle file operations, work with dates, process data efficiently, and create plots. They provide the tools we need for our financial data analysis.

## Read and process option chain data

First, we define a function to read and process option chain data from CSV files

In [ ]:
def read_chains(fl):
    df = (
        pl.read_csv(fl)
          .with_columns([
              pl.col("date").str.to_datetime("%m/%d/%Y", strict=False),
              pl.col("option_expiration").str.to_datetime("%m/%d/%Y", strict=False),
              pl.col("forward_price")
                .cast(pl.Float64, strict=False)
                .fill_null(pl.lit(None).cast(pl.Float64))
          ])
          .sort("date")
    )
    return df

This function reads a CSV file and processes its contents. It converts date columns to datetime format and ensures the forward price is handled correctly. The function then sorts the data by date and returns the processed dataframe.

Now we use this function to read all CSV files in a specific directory

In [ ]:
files = glob.glob(os.path.join("rut-eod", "*.csv"))
df = pl.concat([read_chains(fl) for fl in files], how="vertical_relaxed")

We locate all CSV files in the "rut-eod" directory and read them using our read_chains function. The resulting dataframes are combined into a single large dataframe containing all the option chain data.

## Define helper functions for data analysis

We create two helper functions to extract specific data from our main dataframe

In [ ]:
def read_vol_curve(df, as_of_date, underlying, expiry, delta_low, delta_high):
    return (
        df
        .filter(
            (pl.col("date") == as_of_date)
            & (pl.col("symbol") == underlying)
            & (pl.col("option_expiration") == expiry)
            & (
                ((pl.col("delta") >= delta_low) & (pl.col("delta") <= delta_high))
                | ((pl.col("delta") >= -delta_high) & (pl.col("delta") <= -delta_low))
            )
        )
        .group_by("strike")
        .agg(pl.col("iv").mean().alias("iv_mean"))
        .sort("strike")
    )

In [ ]:
def query_expirations(df, as_of_date, underlying, dte=30):
    cutoff_date = as_of_date + dt.timedelta(days=dte)

    return (
        df
        .filter(
            (pl.col("date") == as_of_date) &
            (pl.col("symbol") == underlying) &
            (pl.col("option_expiration") > cutoff_date)
        )
        .group_by("option_expiration")
        .agg(pl.col("volume").sum().alias("total_volume"))
        .sort("option_expiration")
        .get_column("option_expiration")
        .to_list()
    )

These functions help us extract specific data from our main dataframe. The read_vol_curve function filters and processes data to create a volatility curve, while query_expirations finds relevant expiration dates based on given criteria.

## Set parameters and create volatility smile plot

We set our parameters and create a plot of the volatility smile

In [ ]:
as_of_date = dt.datetime(2013, 6, 3)
expiry = dt.datetime(2015, 12, 18)
underlying = "RUT"
dte = 30
delta_low = 0.05
delta_high = 0.50

In [ ]:
expiries = query_expirations(df, as_of_date, underlying, dte)
fig, ax = plt.subplots(figsize=(10, 6))
cmap = plt.get_cmap("rainbow", len(expiries))
format_kw = {"linewidth": 0.5, "alpha": 0.85}

for i, expiry in enumerate(expiries):
    curve = read_vol_curve(
        df,
        as_of_date, 
        underlying, 
        expiry, 
        delta_low, 
        delta_high
    )
    ax.plot(
        curve["strike"], 
        curve["iv_mean"],
        label=expiry.strftime("%Y-%m-%d"),
        color=cmap(i),
        **format_kw
    )
ax.set_ylabel("implied volatility")
ax.legend(loc="upper right", framealpha=0.7)

We set our analysis parameters, including the date, underlying asset, and delta range. We then query for relevant expiration dates and create a plot. For each expiration, we calculate and plot the volatility curve. This creates a visual representation of the volatility smile across different expiration dates.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.